In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
cd gdrive/My\ Drive/DMIS/Re-Implementation/Transformer_Re-Implementation

/content/gdrive/My Drive/DMIS/Re-Implementation/Transformer_Re-Implementation


In [5]:
ls

config.py                  model_save/                 temp.py
dataset/                   model_save_1e-2/            train_blueprint2.ipynb
Embedding_blueprint.ipynb  preprocess_blueprint.ipynb  train_blueprint.ipynb
embedding.py               process.py                  train_colab.ipynb
model_blueprint.ipynb      __pycache__/                train_local.ipynb
model.py                   README.md                   train.py


In [6]:
!nvidia-smi

Wed Aug 28 02:23:21 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8    24W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## ================== drive mounting ======================
## Train

In [0]:
import model as trf
from process import get_data
from train import *
from config import *

In [0]:
import torch
import torch.nn as nn
from torchtext import data, datasets

In [9]:
#@title
print(BATCH_SIZE, DEVICE_SET)

5 cuda


In [0]:
data_path = "dataset/"
file_name = "train."
save_path = "model_save/transformer_epoch"

In [0]:
SRC, TGT, trn = get_data(data_path + file_name)

In [0]:
pad_idx = TGT.vocab.stoi["<pad>"]

In [0]:
model = trf.Transformer(len(SRC.vocab), len(TGT.vocab), N = 6)
model.cuda()
criterion = nn.NLLLoss(ignore_index = 1)
criterion.cuda()
model_opt = torch.optim.Adam(model.parameters(), lr = .005, betas=(.8, .999), eps = 1e-9)

In [0]:
train_iter = data.Iterator(trn, batch_size = BATCH_SIZE,
                          device = DEVICE_SET,
                          repeat = False)

In [0]:
train_model(epochs, model, criterion, model_opt, train_iter, save_path, print_every)

Iteration Step: 0 Loss per token: 0.806310 per Sec: 0.693288 #(tokens) : 0
Iteration Step: 100 Loss per token: 0.553859 per Sec: 27.826119 #(tokens) : 15
Iteration Step: 200 Loss per token: 0.486413 per Sec: 28.172985 #(tokens) : 16
Iteration Step: 300 Loss per token: 0.490679 per Sec: 27.980104 #(tokens) : 15
Iteration Step: 400 Loss per token: 0.482696 per Sec: 27.575587 #(tokens) : 16
Iteration Step: 500 Loss per token: 0.457818 per Sec: 28.760225 #(tokens) : 16
Iteration Step: 600 Loss per token: 0.463033 per Sec: 28.733830 #(tokens) : 16
Iteration Step: 700 Loss per token: 0.457907 per Sec: 28.360838 #(tokens) : 16
Iteration Step: 800 Loss per token: 0.456539 per Sec: 27.951502 #(tokens) : 16
Iteration Step: 900 Loss per token: 0.458130 per Sec: 28.626301 #(tokens) : 16
Iteration Step: 1000 Loss per token: 0.482287 per Sec: 28.252217 #(tokens) : 15
Iteration Step: 1100 Loss per token: 0.461172 per Sec: 28.422095 #(tokens) : 16
Iteration Step: 1200 Loss per token: 0.461764 per Sec:

In [0]:
model.load_state_dict(torch.load("model_save/transformer_epoch0.pt"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

  
  #(total_epoch) : 0